In [1]:
# Cell 1
# Import necessary libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    precision_recall_fscore_support,
    accuracy_score,
    f1_score,
    classification_report,
)
import numpy as np 



In [2]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    # Fit the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate metrics
    conf_matrix = confusion_matrix(y_test, y_pred)
    precision_recall_f1 = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=1)
    accuracy = accuracy_score(y_test, y_pred)
    macro_avg_f1 = f1_score(y_test, y_pred, average='macro')
    weighted_avg_f1 = f1_score(y_test, y_pred, average='weighted')

    return {
        'y_pred': y_pred,  # Add this line
        'conf_matrix': conf_matrix,
        'precision_recall_f1': precision_recall_f1,
        'accuracy': accuracy,
        'macro_avg_f1': macro_avg_f1,
        'weighted_avg_f1': weighted_avg_f1,
    }


In [3]:
# Cell 3
# Load Penguin Dataset and Prepare Data
penguin_path = r'C:\Users\ibrah\OneDrive\Desktop\COMP472-A1-datasets\penguins.csv'
penguin_data = pd.read_csv(penguin_path)

# Convert features into 1-hot vectors
categorical_columns = ['island', 'sex']
penguin_categorical = penguin_data[categorical_columns]
encoder = OneHotEncoder()
one_hot_encoded = encoder.fit_transform(penguin_categorical).toarray()
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
penguin_data_encoded = pd.concat([penguin_data, one_hot_df], axis=1).drop(categorical_columns, axis=1)

# Split the dataset
penguin_features = penguin_data_encoded.drop('species', axis=1)
penguin_target = penguin_data_encoded['species']
penguin_X_train, penguin_X_test, penguin_y_train, penguin_y_test = train_test_split(
    penguin_features, penguin_target, test_size=0.2, random_state=42
)


In [4]:
# Cell 4
# Define models and their hyperparameter grids
models = {
    "Base DT": DecisionTreeClassifier(),
    "Top DT": DecisionTreeClassifier(max_depth=5),
    "Base MLP": MLPClassifier(random_state=42, max_iter=1000),
    "Top MLP": MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', hidden_layer_sizes=(30, 50), solver='adam', max_iter=1000),
}


In [5]:
# Cell 5
# Store the test data outside the loop
class_labels = ['Adelie', 'Chinstrap', 'Gentoo']
test_data = (penguin_y_test, class_labels)

# Repeat the process 5 times for each model
performance_results = []

for _ in range(5):
    for model_name, model in models.items():
        # If it's a grid search model, perform the grid search
        if isinstance(model, GridSearchCV):
            model.fit(penguin_X_train, penguin_y_train)
            best_params = model.best_params_
            best_model = model.best_estimator_
        else:
            # For non-grid search models, use the model as is
            best_params = model.get_params()
            best_model = model

        # Evaluate the model
        model_results = evaluate_model(best_model, penguin_X_train, penguin_X_test, penguin_y_train, penguin_y_test)

        # Append the results to the performance list
        performance_results.append({
            'model_name': model_name,
            'best_model': best_model,
            'results': model_results
        })


In [6]:
# Cell 6
# Calculate and print the average and variance for each metric
for metric in ['accuracy', 'macro_avg_f1', 'weighted_avg_f1']:
    metric_values = [result['results'][metric] for result in performance_results if metric in result['results']]
    average_metric = np.mean(metric_values)
    variance_metric = np.var(metric_values)

    print(f"Avg {metric}: {average_metric}")
    print(f"Variance {metric}: {variance_metric}")
    print()

Avg accuracy: 0.8067164179104477
Variance accuracy: 0.04411840053464023

Avg macro_avg_f1: 0.7269561455426129
Variance macro_avg_f1: 0.0932304917184717

Avg weighted_avg_f1: 0.7627928599327615
Variance weighted_avg_f1: 0.07348463092003907



In [7]:
# Cell 7 (Updated)
# Save the results to a file
with open("penguin_performance.txt", "a") as file:
    for result in performance_results:
        model_name = result['model_name']
        best_model = result['best_model']  # Use the stored best model
        model_results = result['results']

        file.write(f"{model_name} Model:\n")
        file.write("(A) Model Description:\n{} Model with hyperparameters: {}\n".format(model_name, best_model.get_params()))
        file.write("(B) Confusion Matrix:\n{}\n".format(model_results['conf_matrix']))

        # Additional formatting for precision, recall, and f1
        precision, recall, f1, _ = model_results['precision_recall_f1']
        file.write("Precision Scores: {}\n".format(precision))
        file.write("Recall Scores: {}\n".format(recall))
        file.write("F1 Scores: {}\n".format(f1))

        # Calculate the classification report
        report_str = classification_report(
            test_data[0], y_pred=model_results['y_pred'], target_names=class_labels, zero_division=1
        )
        file.write("(C) Precision, Recall, and F1:\n{}\n".format(report_str))

        file.write("(D) Accuracy: {}\n".format(model_results['accuracy']))
        file.write("(D) Macro-average F1: {}\n".format(model_results['macro_avg_f1']))
        file.write("(D) Weighted-average F1: {}\n".format(model_results['weighted_avg_f1']))

        file.write("--------------------\n")  # Separator

# Ensure you close the file to save the changes
file.close()

In [8]:
# Load the dataset
abalone_path = r'C:\Users\ibrah\OneDrive\Desktop\COMP472-A1-datasets\abalone.csv'
abalone_data = pd.read_csv(abalone_path)

# Split the dataset into features and target
abalone_features = abalone_data.drop('Type', axis=1)
abalone_target = abalone_data['Type']

# Split the dataset into training and testing sets
abalone_X_train, abalone_X_test, abalone_y_train, abalone_y_test = train_test_split(
    abalone_features, abalone_target, test_size=0.2, random_state=42
)


In [9]:
# Define the models
models = {
    "Base DT": DecisionTreeClassifier(),
    "Top DT": DecisionTreeClassifier(max_depth=5),
    "Base MLP": MLPClassifier(random_state=42, max_iter=1000),
    "Top MLP": MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', hidden_layer_sizes=(30, 50), solver='adam', max_iter=1000),
}


In [10]:
# Store the test data outside the loop
test_data = (abalone_y_test, ['M', 'F', 'I'])  # The target variable is 'Type'

# Repeat the process 5 times for each model
performance_results = []

for _ in range(5):
    for model_name, model in models.items():
        # If it's a grid search model, perform the grid search
        if isinstance(model, GridSearchCV):
            model.fit(abalone_X_train, abalone_y_train)
            best_params = model.best_params_
            best_model = model.best_estimator_
        else:
            # For non-grid search models, use the model as is
            best_params = model.get_params()
            best_model = model

        # Evaluate the model
        model_results = evaluate_model(best_model, abalone_X_train, abalone_X_test, abalone_y_train, abalone_y_test)

        # Append the results to the performance list
        performance_results.append({
            'model_name': model_name,
            'best_model': best_model,
            'results': model_results
        })


In [11]:
# Calculate and print the average and variance for each metric
for metric in ['accuracy', 'macro_avg_f1', 'weighted_avg_f1']:
    metric_values = [result['results'][metric] for result in performance_results if metric in result['results']]
    average_metric = np.mean(metric_values)
    variance_metric = np.var(metric_values)

    print(f"Avg {metric}: {average_metric}")
    print(f"Variance {metric}: {variance_metric}")
    print()


Avg accuracy: 0.5206937799043062
Variance accuracy: 0.00030893008401822273

Avg macro_avg_f1: 0.5102026952042623
Variance macro_avg_f1: 0.0004946265815827906

Avg weighted_avg_f1: 0.5108571533506223
Variance weighted_avg_f1: 0.0004424166763034035



In [ ]:
# Save the results to a file
with open("abalone_performance.txt", "a") as file:
    for result in performance_results:
        model_name = result['model_name']
        best_model = result['best_model']  # Use the stored best model
        model_results = result['results']

        file.write(f"{model_name} Model:\n")
        file.write("(A) Model Description:\n{} Model with hyperparameters: {}\n".format(model_name, best_model.get_params()))
        file.write("(B) Confusion Matrix:\n{}\n".format(model_results['conf_matrix']))

        # Additional formatting for precision, recall, and f1
        precision, recall, f1, _ = model_results['precision_recall_f1']
        file.write("Precision Scores: {}\n".format(precision))
        file.write("Recall Scores: {}\n".format(recall))
        file.write("F1 Scores: {}\n".format(f1))

        # Calculate the classification report
        report_str = classification_report(
            test_data[0], y_pred=model_results['y_pred'], target_names=test_data[1], zero_division=1
        )
        file.write("(C) Precision, Recall, and F1:\n{}\n".format(report_str))

        file.write("(D) Accuracy: {}\n".format(model_results['accuracy']))
        file.write("(D) Macro-average F1: {}\n".format(model_results['macro_avg_f1']))
        file.write("(D) Weighted-average F1: {}\n".format(model_results['weighted_avg_f1']))

        file.write("--------------------\n")  # Separator

# Ensure you close the file to save the changes
file.close()